# Monte Carlo Radiative Transfer

## Background

### Important Variables

- Specific Intensity $I_v$ or $I(\mathbf{x}, t, \nu)$
- $\mathbf{x}$ position in space
- $\mu$ propagation direction
- $t$ time
- $\nu$ frequency
- $\chi_a$ absorption coefficient
- $\chi_s$ scattering coefficient
- $\chi_t = \chi_a + \chi_s$ total extinction coefficient or opacity

The radiative transfer equation describes the propagation of light through a medium. Let us start by assuming the light propagates through empty space then

$$
\frac{dI_v}{ds} = 0
$$

where $I$ is the intensity of light and $s$ is the distance traveled. Let us add a radiation absorbing medium to the mix. The radiative transfer equation then becomes

$$
\frac{dI_v}{ds} = -\alpha_v I_v
$$

where $\alpha$ is the *fractioanl depletion coefficient* per path length. We can define the optical depth $\tau$ as

$$
d \tau_v = -\alpha_v ds
$$

and the radiative transfer equation becomes

$$
\frac{dI_v}{d\tau_v} = -I_v
$$

However radiation can also travel through a radiation emitting medium. The radiative transfer equation then becomes

$$
\frac{dI_v}{ds} = -\alpha_v I_v + j_v
$$

where $j$ is the *emission coefficient*. We can also define it in terms of the optical depth

$$
\frac{dI_v}{d\tau_v} = \frac{j_v}{\alpha_v} - I_v.
$$

We then define the source function as 

$$
S_v = \frac{j_v}{\alpha_v}
$$

and the radiative transfer equation becomes

$$
\frac{dI_v}{d\tau_v} = S_v - I_v.
$$

## MCRT 

We can simulate the radiative transfer equation using Monte Carlo methods. We leverage an abstraction called photon packets, or machine photons, where a number of photons are grouped together. The photons are described by their position, direction, frequency, energy and weight. Over the course of the simulation, each photon packet continously accumulates optical depth. Hence contrary to classical simulations, we do not integrate over time but over optical depth.



We treat interactions with the medium as discrete interaction events, hence the MC algorithm needs to find when those events occur. We denote a random number as $\xi$. At the beginning of each packet trajectory event, each packet is assigned a new optical depth distance to the next interaction by

$$
\tau = -\log(\xi)
$$

we then compute the length of the path in physical space by dividing the optical depth by the total extinction coefficient $\chi_t$

$$
l = \frac{\tau}{\chi_t}.
$$

We can then update the position of the photon packet by

$$
\mathbf{x} = \mathbf{x} + l \mathbf{\mu}
$$

where $\mathbf{\mu}$ is the direction of the photon packet. We then check if the photon packet has left the medium. If it has, we record the energy of the packet and discard it. If it has not, we check if the photon packet has interacted with the medium. We do this by sampling a random number $\xi$ and comparing it to the probability of interaction

In [1]:
using Plots

In [2]:
# We define the Photon Packets
mutable struct PP
    # position in physical space
    pos::AbstractArray{Float64, 1}
    # propagation direction
    mu::AbstractArray{Float64, 1}
    # state of the packet, False means the photon has escaped
    absorbed::AbstractArray{Bool, 1}
    escaped::AbstractArray{Bool, 1}
end

In [8]:
struct Params
    # scattering coefficient
    chi_scattering::Float64
    # absorbtion coefficient
    chi_absorbtion::Float64
    # boundary of the domain
    boundary::Float64
    # enable scattering -> Drawing new directions
    enable_scattering::Bool
end

function event(p::PP, params::Params)::Nothing

    N = size(p.pos)[1]

    # sample optical depth to next event
    tau_to_next = -log.(rand(Float64, N))

    # set chi total / opacity 
    chi = params.chi_absorbtion
    if params.enable_scattering
        chi += params.chi_scattering
    end
    
    # Invert the optical depth to get the phyiscal distance. 
    # Note that this is an approximation, as we cannot invert the integral over chi as there is no unique solution (?)
    l_to_next = tau_to_next ./ chi

    # Decide if packets crossed boundaries
    escaped = (p.pos .> params.boundary) .|| (p.pos .< -params.boundary)

    # Decide if packets are scattered or absorbed   
    is_absorbed = rand(N) .< (params.chi_absorbtion / (params.chi_absorbtion + params.chi_scattering))
    
    # Update the state of the packets
    p.pos .+= l_to_next .* p.mu

    # Scatter the packets -> Draw new propagation directions
    if params.enable_scattering
        p.mu = rand(Float64, N) .* .2  .- 1
    end

    # Update the state of the packets
    p.absorbed = p.absorbed .|| (is_absorbed .&& .!p.escaped)
    p.escaped = p.escaped .|| (escaped .&& .!p.absorbed)
    
    return nothing
    
end

event (generic function with 1 method)

In [9]:
function mcrt(
    N::Int64, 
    matter_density_profile::Function,
    params::Params,
    ) :: Float64


    println("Initalizing packets...")
    radius = 1.0
    # Initialize the packets
    pp = PP(
        radius .* rand(Float64, N) .^3,
        rand(Float64, N) .* 2 .- 1,
        zeros(Bool, N),
        zeros(Bool, N),
    )

    println("Starting simulation...")
    # Loop over the packets
    while !all((pp.absorbed .|| pp.escaped))
        # println("P absorbed: ", sum(pp.absorbed) / N, " P escaped: ", sum(pp.escaped) / N)
        event(pp, params)
    end

    return sum(pp.escaped) / N
end

mcrt (generic function with 1 method)

In [10]:
for i in 1:10
    esc_prob = mcrt(
        10000,
        x -> 1 / (x + 1e-6)^2,
        Params(
            0.8,
            0.2,
            1.0,
            false
        )
    )

    println("Escape probability: ", esc_prob)
end
    
params = Params(
    0.8,
    0.2,
    1.0,
    false
)

esc_prob = mcrt(
    10000,
    x -> 1 / (x + 1e-6)^2,
    params
)

println("Escape probability: ", esc_prob)

Initalizing packets...
Starting simulation...
Escape probability: 0.3656
Initalizing packets...
Starting simulation...
Escape probability: 0.3747
Initalizing packets...
Starting simulation...
Escape probability: 0.3756
Initalizing packets...
Starting simulation...
Escape probability: 0.3792
Initalizing packets...
Starting simulation...
Escape probability: 0.3735
Initalizing packets...
Starting simulation...
Escape probability: 0.3801
Initalizing packets...
Starting simulation...
Escape probability: 0.3688
Initalizing packets...
Starting simulation...
Escape probability: 0.3742
Initalizing packets...
Starting simulation...
Escape probability: 0.3818
Initalizing packets...
Starting simulation...
Escape probability: 0.3659
Initalizing packets...
Starting simulation...
Escape probability: 0.3737
